In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle

os.chdir('../..')
os.getcwd()

#from graph_creator.MapGraph import MapGraph
#from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

### Start comparing graphs:

In [3]:
from glob import glob
import pickle
import networkx as nx
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

graph_paths = glob("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla/*.pkl")
len(graph_paths)

5292

In [4]:
# Define a random set of coverage situations with 3 nodes each
# This still needs to be replaced by more strategically selected graphs

idxs = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
coverage_graphs = {}
cov_data = {}

for idx in idxs:
    graph_path = graph_paths[0]
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)
    #random.sample(ag_nx.nodes, 3)
    coverage_graphs[idx] = ag_nx.subgraph(random.sample(list(ag_nx.nodes), 3))
    cov_data[idx] = []

cov_data["degree"] = []
cov_data["density"] = []
cov_data["diameter"] = []


In [5]:
for graph_path in graph_paths:
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)

    for key in coverage_graphs.keys():
        GM = nx.algorithms.isomorphism.DiGraphMatcher(
            ag_nx, coverage_graphs[key],
            node_match=nx.algorithms.isomorphism.categorical_node_match('actor_type', None),
            # node_match=nx.algorithms.isomorphism.categorical_node_match(['actor_type', 'lane_change'], [None, None]),
            edge_match=nx.algorithms.isomorphism.categorical_edge_match('edge_type', None)
        )
        cov_data[key].append(GM.subgraph_is_isomorphic())
    
    cov_data["degree"].append(sum(dict(ag_nx.degree()).values()) / len(ag_nx.nodes()))
    cov_data["density"].append(nx.density(ag_nx))
    cov_data["diameter"].append(nx.diameter(ag_nx))

cov_data_df = pd.DataFrame(cov_data)
cov_data_df.mean()

100         0.562358
500         0.562358
1000        0.562358
1500        0.562358
2000        0.562358
2500        0.562358
3000        0.562358
3500        0.562358
4000        0.562358
4500        0.562358
5000        0.562358
degree      3.057126
density     0.658576
diameter    2.931973
dtype: float64

In [ ]:
list(cov_data_df.columns[:10])


In [6]:
cov_data_df.groupby(list(cov_data_df.columns[:11]), as_index=False).size()

,100,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,size
0,False,False,False,False,False,False,False,False,False,False,False,2316
1,True,True,True,True,True,True,True,True,True,True,True,2976


In [ ]:
# sns.pairplot(data=cov_data_df)
# plt.show()


# Notebook junkyard